In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker


In [ ]:
### Read in the data

zl_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/z400_11-15_12.npy')
zl_wave = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/wave/z400_11-15_12.npy')

pvlay_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/pv300-500_11-15_12.npy')
pvlay_wave = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/wave/pv300-500_11-15_12.npy')

slp_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/slp_11-16_12.npy')
slp_wave = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/wave/slp_11-16_12.npy')

p_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/precip-accu_11-16_12.npy')
p_wave = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/wave/precip-accu_11-16_12.npy')
# dp_wave = smooth2d(p_wave - p_ctl,25)
dp_wave = p_wave - p_ctl

lats = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/lats.npy')
lons = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/lons.npy')


In [ ]:
### Plot subdomain
min_lat1 = 29
max_lat1 = 45
min_lon1 = -95
max_lon1 = -79

min_lat2 = 32
max_lat2 = 48
min_lon2 = -78
max_lon2 = -62

min_lat3 = 32
max_lat3 = 48
min_lon3 = -78
max_lon3 = -62

### Cartopy projection
proj = crs.PlateCarree()

### Contour intervals
zlvls = np.arange(0,1500,10)
dzlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.75
dplvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*1.75
dprelvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*6.

fig, axd = plt.subplot_mosaic([['left','middle','right'],['cbar1','cbar2','cbar3']],
                              constrained_layout=True, figsize=(8.0,3.0), dpi=1000, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':proj},
                                              'middle':{'projection':proj},
                                              'right':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['left'].set_extent([min_lon1, max_lon1, min_lat1, max_lat1],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

axd['left'].set_extent([min_lon1, max_lon1, min_lat1, max_lat1],crs=crs.PlateCarree())

cs1 = axd['left'].contour(lons, lats, smooth2d(zl_ctl,2)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['left'].contour(lons, lats, smooth2d(zl_wave,2)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['left'].contourf(lons, lats, pvlay_wave - pvlay_ctl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',zorder=-20,
             transform=crs.PlateCarree())
axd['left'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['left'].set_title('PV at $t$ = 48 h')
axd['left'].text(0.0, 1.0, '(a)',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### MIDDLE
#############

# Add the gridlines
gls = axd['middle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['middle'].set_extent([min_lon2, max_lon2, min_lat2, max_lat2],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

cs1 = axd['middle'].contour(lons, lats, smooth2d(slp_ctl,20),levels=np.arange(0,1500,6),
                colors='k',transform=crs.PlateCarree())
axd['middle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='k')

cs2 = axd['middle'].contour(lons, lats, smooth2d(slp_wave,20),levels=np.arange(0,1500,6),
                colors='magenta',transform=crs.PlateCarree())
axd['middle'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='magenta')

im2 = axd['middle'].contourf(lons, lats, slp_wave - slp_ctl, levels=dplvls,
             cmap=get_cmap('RdBu_r'), extend='both',zorder=-20,
             transform=crs.PlateCarree())
axd['middle'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['middle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['middle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['middle'].set_title('SLP at $t$ = 72 h')
axd['middle'].text(0.0, 1.0, '(b)',transform=axd['middle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### RIGHT
##############

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['right'].set_extent([min_lon3, max_lon3, min_lat3, max_lat3],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

im3 = axd['right'].contourf(lons, lats, dp_wave, levels=dprelvls,
            cmap = get_cmap('BrBG'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['right'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['right'].set_title('Precip through $t$ = 72 h')
axd['right'].text(0.0, 1.0, '(c)',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar1 = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar1'])
cbar1.set_ticklabels(['','-5.25','','-2.25','','0.75','','3.75','','6.75'],fontsize=8)

cbar2 = fig.colorbar(im2, orientation='horizontal', cax=axd['cbar2'])
cbar2.set_ticklabels(['','-12.25','','-5.25','','1.75','','8.75','','15.75'],fontsize=8)

cbar3 = fig.colorbar(im3, orientation='horizontal', cax=axd['cbar3'])
cbar3.set_ticklabels(['','-42','','-18','','6','','30','','54'],fontsize=8)

plt.savefig('/p/work1/lloveras/real/nov2018/figs_nov2018/4km_wave_all.pdf',bbox_inches='tight')
plt.show()
